In [70]:
from odlpet import Compression, Scanner
import read_mCT_data #in odlpet > util
from odlpet.scanner import scanner

In [71]:
import sys
print(sys.version)

3.6.7 | packaged by conda-forge | (default, Nov  6 2019, 16:19:42) 
[GCC 7.3.0]


In [72]:
#scanner_ = scanner.mCT() # incompatible model "ERROR: construct_proj_data_info: max_ring_difference 49 is too large, number of rings is 8"

In [73]:
import my_mct
scanner_ = my_mct.my_mCT() # an alternative, see Shannon
import mct
scanner_ = mct.mCT() # see copiemct

In [74]:
headers = ["CylinderPET-LM-00-sino-0.s.hdr", "CylinderPET-LM-00.hdr", "narc.hdr", "arc.s.hdr"]

In [75]:
verbose = True
automated_compressions = []
for k, hdr in enumerate(headers):
    if verbose:
        print("read file text")
        with open(hdr, 'r') as file: 
            print(file.read())
        print("construct compression object")
    automated_compressions.append(read_mCT_data.compression_from_hdr(scanner_, hdr, compression_obj = None, verbose = verbose))
    if verbose:
        print("disable verbose")
        verbose = False 

read file text
!INTERFILE:=
%comment:=Created from listmode data
!originating system:=1104
%SMS-MI header name space:=sinogram subheader
%SMS-MI version number:=3.5

!GENERAL DATA:=
%listmode header file:=C:\Users\petct\Desktop\CylinderPET-Converted\CylinderPET-LM-00\CylinderPET-LM-00.hdr
%listmode data file:=C:\Users\petct\Desktop\CylinderPET-Converted\CylinderPET-LM-00\CylinderPET-LM-00.l
!name of data file:=CylinderPET-LM-00-sino-0.s

!GENERAL IMAGE DATA:=
%study date (yyyy:mm:dd):=2019:01:25
%study time (hh:mm:ss GMT+00:00):=14:34:48
isotope name:=F-18
isotope gamma halflife (sec):=6586.2
isotope branching factor:=0.9673
radiopharmaceutical:=Fluorodeoxyglucose
%tracer injection date (yyyy:mm:dd):=2019:01:25
%tracer injection time (hh:mm:ss GMT+00:00):=14:32:06
tracer activity at time of injection (Bq):=1e+008
relative time of tracer injection (sec):=151370052
injected volume (ml):=0.0
image data byte order:=LITTLEENDIAN
%patient orientation:=HFS
!PET data type:=emission
data format

In [76]:
import importlib
importlib.reload(read_mCT_data)

<module 'read_mCT_data' from '/home/mamo/Hugo/read_mCT_data.py'>

In [77]:
def compare(x, y, attributes):
    errors = 0
    for name in attributes:
        if getattr(x, name) != getattr(y, name):
            errors = errors + 1
    return errors == 0

In [78]:
manual_compression = Compression(scanner_)
names = ['span_num', 'max_num_segments', 'num_of_views', 'num_non_arccor_bins', 'data_arc_corrected'] 
values = [11, 49, 168, 400, False]
for pair in zip(names, values):
    setattr(manual_compression, *pair)

for k, c in enumerate(automated_compressions):
    is_equal = compare(c, manual_compression, names)
    if is_equal:
        print("success, automated compression matches manual compression ")
    else:
        print("error, attribute mismatch")
    if k == 2: 
        manual_compression.data_arc_corrected = True

success, automated compression matches manual compression 
success, automated compression matches manual compression 
success, automated compression matches manual compression 
success, automated compression matches manual compression 


In [80]:
proj = automated_compressions[0].get_projector()
print('domain\n{} \n\nrange\n{}'.format(proj.domain, proj.range)) # for some reason, the image domain is smaller than Siemens, which has size (109, 200, 200)

domain
uniform_discr([   0., -364., -364.], [ 226.72,  368.  ,  368.  ], (109, 183, 183), dtype='float32') 

range
uniform_discr([   0.,    0., -421.], [ 621.    ,    3.1416,  421.    ], (621, 168, 400), dtype='float32')
